In [ ]:
#import the necessary libraries

import os # navigating through the folders
import numpy as np 
import pandas as pd 
import matplotlib.image as mpimg
from skimage import io
from scipy import ndimage
import cv2
import gc
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Conv2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau

In [ ]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
#reading the train_classes and the submission sample

train_classes = pd.read_csv("../input/planets-dataset/planet/planet/train_classes.csv")
sample_submission = pd.read_csv("../input/planets-dataset/planet/planet/sample_submission.csv")

In [ ]:
# paths to all the images

train_path = ('../input/planets-dataset/planet/planet/train-jpg')
test_path = ('../input/planets-dataset/planet/planet/test-jpg')
test_additional_path = ('../input/planets-dataset/test-jpg-additional/test-jpg-additional')

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

#using the lambda function to create our labels
labels = list(set(flatten([l.split(' ') for l in train_classes['tags'].values])))

#Creating our label map
label_map = {l: i for i, l in enumerate(labels)}


In [ ]:
from itertools import chain
labels_list = list(chain.from_iterable([tags.split(" ") for tags in train_classes['tags'].values]))
labels_set = set(labels_list)
print("There is {} unique labels including {}".format(len(labels_set), labels_set))

In [ ]:
# converting the tags columns into one_hot_vectors
for tag in labels_set:
    train_classes[tag] = train_classes['tags'].apply(lambda x: 1 if tag in x.split() else 0)
    
# adding '.jpg' extension to 'image_name'
train_classes['image_name'] = train_classes['image_name'].apply(lambda x: '{}.jpg'.format(x)) 
train_classes.head()

In [ ]:
y_col = list(train_classes.columns[2:]) 
train_image_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,validation_split=0.2)

# creating the training set
training_set = train_image_gen.flow_from_dataframe(dataframe=train_classes, directory=train_path, x_col='image_name', y_col=y_col, \
       target_size=(224,224), class_mode='raw', seed=0, batch_size=128, subset='training')

# creating the validation set
validation_set = train_image_gen.flow_from_dataframe(dataframe=train_classes, directory=train_path, x_col='image_name', y_col=y_col, \
       target_size=(224,224), class_mode='raw', seed=0, batch_size=128, subset='validation')

In [ ]:
from keras import backend as K
# defining a function to help calculate the fbeta_score
def fbeta_score_K(y_true, y_pred):
    beta_squared = 4

    tp = K.sum(y_true * y_pred) + K.epsilon()
    fp = K.sum(y_pred) - tp
    fn = K.sum(y_true) - tp

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    result = (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

    return result

In [ ]:
import tensorflow.keras as keras
# Loading the pre-trained VGG16 architecture module
from tensorflow.keras.applications.vgg16 import VGG16

# Extract the pre - trained architecture
vgg_model = VGG16(input_shape =(224,224,3),include_top =False, weights ='imagenet')
vgg_model.summary()

# Get the output of the base_model formed above
x = vgg_model.output
x = Flatten()(x)
x = Dense (1000 , activation ='relu')(x)
predictions = Dense (17 , activation ='sigmoid')(x)
model = Model(inputs= vgg_model.input,outputs = predictions)

In [ ]:
history = History()
callbacks = [ EarlyStopping(monitor='val_loss',
                           patience=2,
                           verbose=0)]
             

# setting step size for training and validation image data
step_train_size = int(np.ceil(training_set.samples / training_set.batch_size))
step_val_size = int(np.ceil(validation_set.samples / validation_set.batch_size))

#compiling the model
model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy', fbeta_score_K])

# fitting the model
history = model.fit(x=training_set, steps_per_epoch=step_train_size, validation_data=validation_set, validation_steps=step_val_size,
         epochs=2, callbacks=callbacks)

In [ ]:
#making a copy of the sample submission
sample_copy = sample_submission.copy()
sample_copy['image_name'] = sample_copy['image_name'].apply(lambda x: '{}.jpg'.format(x))
sample_copy.head()


# creating a dataframe for the first 40669 images since we have 2 files for the testing set
test1 = sample_copy.iloc[:40669]['image_name'].reset_index().drop('index', axis=1)
test1.head()

In [ ]:
test_image_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

# generating the image data for the first 40669 images in the sample submission dataframe
test_gen = test_image_gen.flow_from_dataframe(dataframe=test1, \
            directory=test_path, x_col='image_name', y_col=None, \
            batch_size=128, shuffle=False, class_mode=None, target_size=(224,224))


step_test_size1 = int(np.ceil(test_gen.samples / test_gen.batch_size))

In [ ]:
test_gen.reset() # reseting the generator to be sure of avoiding shuffling
pred = model.predict(test_gen, steps=step_test_size1, verbose=1) # predicts the first 40669 images in the 
                                                                    # sample submission dataframe

In [ ]:
unique_labels = set()
def append_labels(tags):
    for tag in tags.split():
        unique_labels.add(tag)

train_classes = train_classes.copy()
train_classes['tags'].apply(append_labels)
unique_labels = list(unique_labels)
print(unique_labels)

In [ ]:
test_file_names1 = test_gen.filenames 
# converting the predictions of the first 40669 to tag names
pred_tags = pd.DataFrame(pred)
pred_tags = pred_tags.apply(lambda x: ' '.join(np.array(unique_labels)[x > 0.5]), axis=1)

# converting the predictions of the first 40669 to a dataframe
import pandas as pd
result1 = pd.DataFrame({'image_name': test_file_names1, 'tags': pred_tags})
print(result1.head())

In [ ]:
# selecting the remaining 'image_name'(s) from the submission_sample dataframe to generate image data from 
# test_additional_path
test_additional = sample_copy.iloc[40669:]['image_name'].reset_index().drop('index', axis=1)
test_additional.head()

In [ ]:
# initializing an image data generator object for the remaining images in the sample submission dataframe
test_image_gen2 = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

# generating the image data for the remaining images in the sample submission dataframe
test_gen2 = test_image_gen2.flow_from_dataframe(dataframe=test_additional, \
            directory=test_additional_path, x_col='image_name', \
            y_col=None, batch_size=128, shuffle=False, class_mode=None, target_size=(224,224))

# setting the step size for the testing set for the remaining images in the sample submission dataframe
step_test_size2 = int(np.ceil(test_gen2.samples / test_gen2.batch_size))


test_gen2.reset() # reseting the generator to be sure of avoiding shuffling
pred2 = model.predict(test_gen2, steps=step_test_size2, verbose=1)

In [ ]:
test_file_names2 = test_gen2.filenames 
        
# converting the predictions of the remaining images to tag names
pred_tags2 = pd.DataFrame(pred2)
pred_tags2 = pred_tags2.apply(lambda x: ' '.join(np.array(unique_labels)[x > 0.5]), axis=1)

# converting the predictions of the remaining to a dataframe
result2 = pd.DataFrame({'image_name': test_file_names2, 'tags': pred_tags2})

In [ ]:
# joining the result of the test data and the additional files
final_result = pd.concat([result1, result2]) 
final_result = final_result.reset_index().drop('index', axis=1) # reseting the index of the dataframe so it 
                                                                # matches that of sample submis

In [ ]:
# removing the jpegs extension back from the dataframe
final_result['image_name'] = final_result['image_name'].apply(lambda x: x[:-4])
final_result.head()

In [ ]:
final_result.to_csv('submission.csv', index=False)